# Semantic Kernel với Tích hợp Máy chủ OpenBnB MCP

Notebook này trình bày cách sử dụng Semantic Kernel với máy chủ OpenBnB MCP thực tế để tìm kiếm chỗ ở Airbnb thật bằng cách sử dụng MCPStdioPlugin. Để truy cập LLM, nó sử dụng Microsoft Foundry. Để thiết lập biến môi trường của bạn, bạn có thể theo hướng dẫn [Bài học Thiết lập](/00-course-setup/README.md)


## Nhập các Gói Cần Thiết


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Tạo Kết Nối Plugin MCP

Chúng ta sẽ kết nối với [Máy chủ OpenBnB MCP](https://github.com/openbnb-org/mcp-server-airbnb) sử dụng MCPStdioPlugin. Máy chủ này cung cấp chức năng tìm kiếm Airbnb thông qua gói @openbnb/mcp-server-airbnb.


## Tạo Client

Trong ví dụ này, chúng ta sẽ sử dụng Microsoft Foundry để truy cập LLM của mình. Hãy đảm bảo các biến môi trường của bạn được thiết lập đúng cách.


## Cấu hình môi trường

Cấu hình các thiết lập Azure OpenAI. Đảm bảo bạn đã đặt các biến môi trường sau:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Hiểu về Tích hợp OpenBnB MCP

Notebook này kết nối với **máy chủ OpenBnB MCP thực tế** cung cấp chức năng tìm kiếm Airbnb thật.

### Cách hoạt động:

1. **MCPStdioPlugin**: Sử dụng giao tiếp đầu vào/đầu ra tiêu chuẩn với máy chủ MCP
2. **Gói NPM Thật**: Tải xuống và chạy `@openbnb/mcp-server-airbnb` thông qua npx
3. **Dữ liệu Thực**: Trả về dữ liệu bất động sản thực từ API của Airbnb
4. **Khám phá Hàm**: Agent tự động khám phá các hàm có sẵn từ máy chủ MCP

### Các Hàm Có Sẵn:

Máy chủ OpenBnB MCP thường cung cấp:
- **search_listings** - Tìm kiếm bất động sản Airbnb theo vị trí và tiêu chí
- **get_listing_details** - Lấy thông tin chi tiết về các bất động sản cụ thể
- **check_availability** - Kiểm tra tình trạng sẵn có cho các ngày cụ thể
- **get_reviews** - Lấy đánh giá cho các bất động sản
- **get_host_info** - Lấy thông tin về chủ nhà của bất động sản

### Yêu cầu tiên quyết:

- **Node.js** đã được cài đặt trên hệ thống của bạn
- **Kết nối Internet** để tải gói máy chủ MCP
- **NPX** có sẵn (đi kèm với Node.js)

### Kiểm tra Kết nối:

Bạn có thể kiểm tra máy chủ MCP thủ công bằng cách chạy:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Lệnh này sẽ tải xuống và khởi động máy chủ OpenBnB MCP, sau đó Semantic Kernel sẽ kết nối để lấy dữ liệu Airbnb thực.


## Chạy Agent với OpenBnB MCP Server

Bây giờ chúng ta sẽ chạy AI Agent kết nối với máy chủ OpenBnB MCP để tìm kiếm các chỗ ở Airbnb thực tế ở Stockholm cho 2 người lớn và 1 trẻ em. Bạn có thể thay đổi danh sách `user_inputs` để sửa đổi tiêu chí tìm kiếm.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Tóm tắt
Chúc mừng! Bạn đã xây dựng thành công một đại lý AI tích hợp với tìm kiếm chỗ ở thực tế sử dụng Giao thức Ngữ cảnh Mô hình (MCP):

## Công nghệ đã sử dụng:
- Semantic Kernel - Để xây dựng đại lý thông minh với Azure OpenAI
- Microsoft Foundry - Cho khả năng LLM và hoàn thành trò chuyện
- MCP (Model Context Protocol) - Để tích hợp công cụ tiêu chuẩn hóa
- Máy chủ OpenBnB MCP - Cho chức năng tìm kiếm Airbnb thực tế
- Node.js/NPX - Để chạy máy chủ MCP bên ngoài

## Những gì bạn đã học:
- Tích hợp MCP: Kết nối các đại lý Semantic Kernel với máy chủ MCP bên ngoài
- Truy cập dữ liệu thời gian thực: Tìm kiếm các bất động sản Airbnb thực tế qua API trực tiếp
- Giao tiếp theo giao thức: Sử dụng giao tiếp stdio giữa đại lý và máy chủ MCP
- Phát hiện chức năng: Tự động phát hiện các chức năng có sẵn từ các máy chủ MCP
- Phản hồi phát trực tuyến: Thu thập và ghi lại các cuộc gọi chức năng theo thời gian thực
- Kết xuất HTML: Định dạng phản hồi đại lý với bảng có kiểu dáng và hiển thị tương tác

## Bước tiếp theo:
- Tích hợp thêm các máy chủ MCP khác (thời tiết, chuyến bay, nhà hàng)
- Xây dựng hệ thống đa đại lý kết hợp giao thức MCP và A2A
- Tạo các máy chủ MCP tùy chỉnh cho nguồn dữ liệu của riêng bạn
- Triển khai bộ nhớ hội thoại liên tục qua các phiên
- Triển khai đại lý lên Azure Functions với điều phối máy chủ MCP
- Thêm xác thực người dùng và khả năng đặt chỗ


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Tuyên bố từ chối trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ nguyên bản nên được coi là nguồn tham khảo chính thức. Đối với thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp do con người thực hiện. Chúng tôi không chịu trách nhiệm về bất kỳ sự hiểu nhầm hoặc giải thích sai nào phát sinh từ việc sử dụng bản dịch này.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
